In [1]:
import json
import os
import time
from datetime import datetime

from os.path import dirname, realpath
import sys
sys.path.append(dirname(dirname(realpath(os.getcwd()))))
import pickle
import torch
import multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from collections import Counter
from copy import deepcopy
import datetime
import tqdm
import numpy as np
import random
import pandas as pd
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

from collections import Counter
sys.path.append("./src/")

testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
from sklearn.metrics import confusion_matrix

import datetime

def parse_date(date_str):
    if date_str == 'NA' or date_str is None:
        return datetime.datetime(9999,1,1,0,0)
    else:
        if len(date_str) == 10:
            format_str = '%Y-%m-%d'
        else:
            raise Exception("Format for {} not recognized!".format(date_str))
    return datetime.datetime.strptime(date_str, format_str)


In [2]:
VTE_path = 'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei'

In [3]:
test_path = os.path.join(VTE_path, 'A2_CMP\\train\\train.json')
X = json.load(open(os.path.join(test_path), 'r'))


In [19]:
m = testDF[["patient_id", "index_date","diag_date", "obs_time_end",  "outcome","outcome_date" ]]

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]
m1 = pd.concat([m_VE1, m_VE0, m0])
m1['PatICN'] =  m1.patient_id.astype(str)

m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
m_VE0['PatICN'] =  m_VE0.patient_id.astype(str)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_71704\1629420586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_71704\1629420586.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_71704\1629420586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a 

In [21]:
len( X.keys())

74188

In [22]:
Y = X

In [23]:

for p in Y.keys():
    
    Y[p]['dxdate'] =  m1 [ m1 ['PatICN']==p ]['diag_date'].values[0]

    if sum( p  ==  m_VE0.PatICN ) ==1 :
        m_p = m_VE0[  p  ==  m_VE0.PatICN ] 
        Yp = Y[p]['events']

        m_date = m_p['model_date'].astype('datetime64[ns]')
        Yp_f = filter(lambda x: parse_date(x['admdate']) < m_date.ravel()[0] , Yp)
        Yp_new = list( Yp_f)
        Y[p]['events']  = Yp_new 
    
    

In [24]:
os.mkdir(os.path.join(VTE_path, 'A2_CMP\\train2'))

In [25]:
adjtrain = os.path.join(VTE_path, 'A2_CMP\\train2\\train.json')


In [26]:
with open(adjtrain, 'w') as fwd:
    print("Saving to {}\n".format(adjtrain))
    json.dump(Y, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\train2\train.json

